In [1]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

/home/mathys/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [3]:
ee.Initialize(project="ee-sentinel-analysis")

Collection : https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD

In [5]:
# Load the Sentinel-1 ImageCollection, filter to Jun-Sep 2020 observations.
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(
    '2020-06-01', '2020-10-01'
)

# Filter the Sentinel-1 collection by metadata properties.
vv_vh_iw = (
    sentinel_1.filter(
        # Filter to get images with VV and VH dual polarization.
        ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
    )
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(
        # Filter to get images collected in interferometric wide swath mode.
        ee.Filter.eq('instrumentMode', 'IW')
    )
)

# Separate ascending and descending orbit images into distinct collections.
vv_vh_iw_asc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
)
vv_vh_iw_desc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
)

# Calculate temporal means for various observations to use for visualization.
# Mean VH ascending.
vh_iw_asc_mean = vv_vh_iw_asc.select('VH').mean()
# Mean VH descending.
vh_iw_desc_mean = vv_vh_iw_desc.select('VH').mean()
# Mean VV for combined ascending and descending image collections.
vv_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VV').mean()
# Mean VH for combined ascending and descending image collections.
vh_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VH').mean()

# Display the temporal means for various observations, compare them.
m = geemap.Map()
m.add_layer(vv_iw_asc_desc_mean, {'min': -12, 'max': -4}, 'vv_iw_asc_desc_mean')
m.add_layer(vh_iw_asc_desc_mean, {'min': -18, 'max': -10}, 'vh_iw_asc_desc_mean')
m.add_layer(vh_iw_asc_mean, {'min': -18, 'max': -10}, 'vh_iw_asc_mean')
m.add_layer(vh_iw_desc_mean, {'min': -18, 'max': -10}, 'vh_iw_desc_mean')

beauvais_roi = [
    [2.470013829667166, 48.49155618695181],
    [2.4864504065104276, 48.49155618695181],
    [2.4864504065104276, 48.49718725242921],
    [2.470013829667166, 48.49718725242921],
    [2.470013829667166, 48.49155618695181]
]

# Calculate the centroid (average of lat and lon)
centroid_lon = sum([coord[0] for coord in beauvais_roi]) / len(beauvais_roi)
centroid_lat = sum([coord[1] for coord in beauvais_roi]) / len(beauvais_roi)

# Set the map center using the calculated centroid
m.setCenter(centroid_lon, centroid_lat, zoom=15)

# Add the polygon to the map as a layer
style = {
    'color': 'red',      # Border color
    'fillColor': '00000000',  # Fill color (transparent)
    'width': 3           # Border width
}
polygon_roi = ee.Geometry.Polygon(beauvais_roi)
m.addLayer(polygon_roi, style, "Beauvais")

m

Map(center=[48.49380861314277, 2.4765884604044706], controls=(WidgetControl(options=['position', 'transparent_…

In [6]:
vv_vh_iw_millis = vv_vh_iw.aggregate_array("system:time_start").getInfo()

vv_vh_iw_timestamp = [datetime.datetime.fromtimestamp(x // 1000).strftime('%Y-%m-%d') for x in vv_vh_iw_millis]

In [7]:
vv_vh_iw_timestamp = pd.Series(vv_vh_iw_timestamp).unique()

In [8]:
vv_vh_iw_timestamp[0]

'2020-06-01'

In [9]:
vv_vh_iw.filterDate(vv_vh_iw_timestamp[0]).mean()

In [10]:
test_export = vv_vh_iw.filterDate(vv_vh_iw_timestamp[0], vv_vh_iw_timestamp[1]).mean()

In [11]:
image_bands = test_export.select(["VV", "VH"])
sampled_image = image_bands.sample(region=polygon_roi, scale=30, numPixels=5000)

In [12]:
pixels = np.array([feature['properties'] for feature in sampled_image['features']])

vv_band = np.array([v['VV'] for v in pixels])
vh_band = np.array([v['VH'] for v in pixels])

TypeError: 'FeatureCollection' object is not subscriptable

In [13]:
task = ee.batch.Export.image.toDrive(
    image=image_bands,
    description='image_export_crstransform',
    folder='.',
    region=polygon_roi,
    crsTransform=[30, 0, -2493045, 0, -30, 3310005],
    crs='EPSG:5070'
)
task.start()
